In [1]:
import os 
os.chdir("../")
%pwd

'/Users/ruchirkanthgandikota/Documents/ML Projects/Wine Quality/end-to-end-mlproject'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class IngestionConfig:
    root_dir: Path 
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml, create_directories


In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> IngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = IngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from mlproject import logger
from mlproject.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: IngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [ ]:
# Pipeline - Flow of execution
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-02-12 13:14:57,539: INFO: common: yaml file: config/config.yml loaded successfully]
[2025-02-12 13:14:57,541: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-12 13:14:57,546: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-02-12 13:14:57,547: INFO: common: created directory at: artifacts]
[2025-02-12 13:14:57,549: INFO: common: created directory at: artifacts/data_ingestion]
[2025-02-12 13:14:57,842: INFO: 24094321: artifacts/data_ingestion/data.zip download! with following info: 
X-GUploader-UploadID: AHMx-iFdRQP8Lx-DRx3nPCv3pTnI5CzpKzmNhVbeJw7SmzaRTirQHR3EInJq53LgYb4iSALK
Expires: Wed, 12 Feb 2025 18:14:57 GMT
Date: Wed, 12 Feb 2025 18:14:57 GMT
Cache-Control: private, max-age=0
Last-Modified: Sat, 21 Sep 2019 19:06:16 GMT
ETag: "055e79eac2d90b1f7031be4d9d39d379"
x-goog-generation: 1569092776878130
x-goog-metageneration: 1
x-goog-stored-content-encoding: identity
x-goog-stored-content-length: 26176
Content-Type: application/zip
x-goog-hash: cr